In [6]:
# load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd


from pytorch_forecasting import Baseline, DeepAR, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.data.examples import generate_ar_data
from pytorch_forecasting.metrics import MAE, SMAPE, MultivariateNormalDistributionLoss

c:\users\jeremy tan\appdata\local\programs\python\python38\lib\site-packages\pytorch_forecasting\models\base_model.py:30: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Pytorch Forecasting Website: https://pytorch-forecasting.readthedocs.io/en/stable/index.html

In [2]:
df = pd.read_csv("2021-2022 data.csv")

In [4]:
df = df[df.columns[1:]]

In [25]:
df["time_idx"] = np.arange(len(df))
df["static"] = "2" # static column
df["series"] = 0
df

,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,datetime,time_idx,static,series
0,0.0,0.0,0.0,0.0,0.0,12.72,2021-01-01 00:00:00,0,2,0
1,0.0,0.0,0.0,0.0,0.0,12.71,2021-01-01 01:00:00,1,2,0
2,0.0,0.0,0.0,0.0,0.0,12.73,2021-01-01 02:00:00,2,2,0
3,0.0,0.0,0.0,0.0,0.0,12.70,2021-01-01 03:00:00,3,2,0
4,0.0,0.0,0.0,0.0,0.0,12.68,2021-01-01 04:00:00,4,2,0
...,...,...,...,...,...,...,...,...,...,...
17515,0.0,0.0,0.0,0.0,0.0,12.11,2022-12-31 19:00:00,17515,2,0
17516,0.0,0.0,0.0,0.0,0.0,12.11,2022-12-31 20:00:00,17516,2,0
17517,0.0,0.0,0.0,0.0,0.0,12.11,2022-12-31 21:00:00,17517,2,0
17518,0.0,0.0,0.0,0.0,0.0,12.09,2022-12-31 22:00:00,17518,2,0


In [34]:
# create dataset and dataloaders
max_encoder_length = 24
max_prediction_length = 24

training_cutoff = df["time_idx"].max() - max_prediction_length

context_length = max_encoder_length
prediction_length = max_prediction_length

training = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Waterlevel_Sto_Nino",
    categorical_encoders={"series": NaNLabelEncoder().fit(df.series)},
    group_ids=["series"],
    static_categoricals=[
        "static"
    ],  # as we plan to forecast correlations, it is important to use series characteristics (e.g. a series identifier)
    time_varying_unknown_reals=["Waterlevel_Sto_Nino"],
    max_encoder_length=context_length,
    max_prediction_length=prediction_length,
)

validation = TimeSeriesDataSet.from_dataset(training, df, min_prediction_idx=training_cutoff + 1)
batch_size = 8
# synchronize samples in each batch over time - only necessary for DeepVAR, not for DeepAR
train_dataloader = training.to_dataloader(
    train=True, batch_size=batch_size, num_workers=0, batch_sampler="synchronized"
)
val_dataloader = validation.to_dataloader(
    train=False, batch_size=batch_size, num_workers=0, batch_sampler="synchronized"
)

c:\users\jeremy tan\appdata\local\programs\python\python38\lib\site-packages\pytorch_forecasting\data\samplers.py:86: UserWarning: Less than 8 samples available for 17449 prediction times. Use batch size smaller than 8. First 10 prediction times with small batch sizes: [25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
  warnings.warn(


In [35]:
validation

TimeSeriesDataSet[length=1](
	time_idx='time_idx',
	target='Waterlevel_Sto_Nino',
	group_ids=['series'],
	weight=None,
	max_encoder_length=24,
	min_encoder_length=24,
	min_prediction_idx=17496,
	min_prediction_length=24,
	max_prediction_length=24,
	static_categoricals=['static'],
	static_reals=[],
	time_varying_known_categoricals=[],
	time_varying_known_reals=[],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['Waterlevel_Sto_Nino'],
	variable_groups={},
	constant_fill_strategy={},
	allow_missing_timesteps=False,
	lags={},
	add_relative_time_idx=False,
	add_target_scales=False,
	add_encoder_length=False,
	target_normalizer=EncoderNormalizer(
	method='standard',
	center=True,
	max_length=None,
	transformation='relu',
	method_kwargs={}
),
	categorical_encoders={'series': NaNLabelEncoder(add_nan=False, warn=True), '__group_id__series': NaNLabelEncoder(add_nan=False, warn=True), 'static': NaNLabelEncoder(add_nan=False, warn=True)},
	scalers={},
	randomize_length=None,
	